In [1]:
!pip install /kaggle/input/lightgbm-3-3-5/lightgbm-3.3.5-py3-none-manylinux1_x86_64.whl

Processing /kaggle/input/lightgbm-3-3-5/lightgbm-3.3.5-py3-none-manylinux1_x86_64.whl
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.2.0
    Uninstalling lightgbm-4.2.0:
      Successfully uninstalled lightgbm-4.2.0


In [2]:
##ordinalgbt code
"""
BSD 3-Clause License

Copyright (c) 2023, adamingas

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products derived from
   this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE."""

from functools import wraps
from lightgbm import LGBMRegressor
import numpy as np
from scipy.optimize import minimize
from sklearn.preprocessing import PolynomialFeatures

def dec_clip_y_pred(fun):
    @wraps(fun)
    def wrapped(*, y_true, y_preds, theta):
        y_preds = np.clip(y_preds, max(theta)-36, a_max=700 + min(theta))
        return fun(y_true=y_true, y_preds=y_preds, theta=theta)

    return wrapped


def stack_zeros_ones(a: np.ndarray, only_zeros=False) -> np.ndarray:
    """Stacks zeroes and ones on the left and rights part of the array
    Stacks horizontally zeros and ones on the left and right hand side of the array
    respectively. If only_zeros is true then it only stacks zeros. This is for the
    gradient which is zero at both ends of the sigmoid.
    e.g.::

        a = [[1,2],
            [3,4],
            [5,6]]
        returns
            [[0,1,2,1],
            [0,3,4,1],
            [0,5,6,1]]


    Parameters
    ----------
    a: np.ndarray
        A 2D array to pad with zeroes and ones
    only_zeros: bool, default False
        If true, then it pads with only zeroes

    Returns
    -------
    np.ndarray

    """
    if only_zeros:
        return np.hstack(
            (
                np.zeros(a.shape[0])[:, np.newaxis],
                a,
                np.zeros(a.shape[0])[:, np.newaxis],
            )
        )

    return np.hstack(
        (np.zeros(a.shape[0])[:, np.newaxis], a, np.ones(a.shape[0])[:, np.newaxis])
    )


def sigmoid(z)->np.ndarray:
    """Sigmoid
    Sigmoid implementation in numpy

    Parameters
    ----------
    z : np.ndarray

    Returns
    -------
    np.ndarray
        Sigmoid
    """
    return 1 / (1 + np.exp(-z))


def grad_sigmoid(z) -> np.ndarray:
    """Gradient
    Gradient of the sigmoid

    Parameters
    ----------
    z : np.ndarray

    Returns
    -------
    np.ndarray
        Gradient of Sigmoid

    """
    phat = sigmoid(z)
    return phat * (1 - phat)


def hess_sigmoid(z) -> np.ndarray:
    """Hessian
    Hessian of the sigmoid
    Sigmoid implementation in numpy

    Parameters
    ----------
    z : np.ndarray

    Returns
    -------
    np.ndarray
        Hessian of sigmoid

    """
    grad = grad_sigmoid(z)
    sig = sigmoid(z)
    return grad * (1 - sig) - sig * (grad)


def alpha2theta(alpha):  # theta[t] = theta[t-1] + exp(alpha[t])
    return np.cumsum(np.append(alpha[0], np.exp(alpha[1:])))


def theta2alpha(theta):  # alpha[t] = log(theta[t] - theta[t-1])
    return np.append(theta[0], np.log(theta[1:] - theta[:-1]))


def probas_from_y_pred(y_preds, theta):
    """
    convers y_preds to probabilities
    """
    s_array: np.ndarray = sigmoid(theta - y_preds[:, np.newaxis])
    # Adding boundary terms of 1 and 0 to make sure that we have probabilities for
    # all classes :TODO: Explain in detail
    # Cumulative probabilities, for column k and row i this matrix represents
    # P(y_i<=k)
    c_probas = stack_zeros_ones(s_array)

    probas = c_probas[:, 1 : len(theta) + 2] - c_probas[:, 0 : len(theta) + 1]
    return probas

@dec_clip_y_pred
def ordinal_logistic_nll(y_true: np.ndarray, y_preds: np.ndarray, theta: np.ndarray):
    """Ordinal Negative log lilelihood

    Parameters
    ----------
    y_true : np.ndarray
        1-D array with correct labels, starts from 0 and goes up to the number
        of unique classes minus one (so unique values are 0,1,2 when dealing
        with three classes)
    y_preds : np.ndarray
        1-D array with predictions in latent space
    theta : np.ndarray
        thresholds, 1-D array, size is the number of classes minus one.

    Returns
    -------
    np.ndarray
        logistic ordinal negative log likelihood

    """
    probas = probas_from_y_pred(y_preds, theta)
    # probabilities associated with the correct label
    label_probas = probas[np.arange(0, len(y_true)), y_true]
    label_probas = np.clip(
        label_probas,
        a_min=np.finfo(float).eps,
        a_max=1 - len(theta) * np.finfo(float).eps
    )
    # loss
    return -np.sum(np.log(label_probas))


# Gradient
def gradient_ordinal_logistic_nll(
    y_true: np.ndarray, y_preds: np.ndarray, theta: np.ndarray
) -> np.ndarray:
    """Gradient of ordinal nll
    Gradient of the ordinal logistic regression with respect to the predictions

    Parameters
    ----------
    y_true : np.ndarray
        1-D array with correct labels, starts from 0 and goes up to the number
        of unique classes minus one (so unique values are 0,1,2 when dealing
        with three classes)
    y_preds : np.ndarray
        1-D array with predictions in latent space
    theta : np.ndarray
        thresholds, 1-D array, size is the number of classes minus one.

    Returns
    -------
    np.ndarray
        Gradient of logistic ordinal negative log likelihood

    """
    y_preds = np.clip(y_preds, -20, a_max=700 + min(theta))
    probas = probas_from_y_pred(y_preds, theta)
    y_true = y_true.astype(int)
    gs_array: np.ndarray = grad_sigmoid(theta - y_preds[:, np.newaxis])
    gc_probas = stack_zeros_ones(gs_array, only_zeros=True)
    g_probas = -(gc_probas[:, 1 : len(theta) + 2] - gc_probas[:, 0 : len(theta) + 1])
    gradient = -(g_probas / probas)[np.arange(0, len(y_true)), y_true]
    return gradient


def hessian_ordinal_logistic_nll(
    y_true: np.ndarray, y_preds: np.ndarray, theta: np.ndarray
) -> np.ndarray:
    """Hessian of ordinal nll
    Hessian of the ordinal logistic regression with respect to the predictions

    Parameters
    ----------
    y_true : np.ndarray
        1-D array with correct labels, starts from 0 and goes up to the number
        of unique classes minus one (so unique values are 0,1,2 when dealing
        with three classes)
    y_preds : np.ndarray
        1-D array with predictions in latent space
    theta : np.ndarray
        thresholds, 1-D array, size is the number of classes minus one.

    Returns
    -------
    np.ndarray
        Hessian of logistic ordinal negative log likelihood

    """
    y_preds = np.clip(y_preds, -20, a_max=700 + min(theta))
    probas = probas_from_y_pred(y_preds, theta)

    y_true = y_true.astype(int)
    gs_array: np.ndarray = grad_sigmoid(theta - y_preds[:, np.newaxis])
    gc_probas = stack_zeros_ones(gs_array, only_zeros=True)
    hs_array: np.ndarray = hess_sigmoid(theta - y_preds[:, np.newaxis])
    hc_probas = stack_zeros_ones(hs_array, only_zeros=True)
    g_probas = -(gc_probas[:, 1 : len(theta) + 2] - gc_probas[:, 0 : len(theta) + 1])
    h_probas = hc_probas[:, 1 : len(theta) + 2] - hc_probas[:, 0 : len(theta) + 1]

    hessian = -(h_probas / probas - np.power(g_probas / probas, 2))[
        np.arange(0, len(y_true)), y_true
    ]
    # hessian[np.abs(hessian) <=np.finfo(float).eps] = -np.finfo(float).eps
    return hessian


def lgb_ordinal_loss(
    y_true: np.ndarray, y_pred: np.ndarray, theta: np.ndarray
):
    """Ordinal loss for lightgbm use
    The ordinal loss used in the lightgbm framework. Returns the
    gradient and hessian of the loss.

    Parameters
    ----------
    y_true : np.ndarray
        1-D array with correct labels, starts from 0 and goes up to the number
        of unique classes minus one (so unique values are 0,1,2 when dealing
        with three classes)
    y_preds : np.ndarray
        1-D array with predictions in latent space
    theta : np.ndarray
        thresholds, 1-D array, size is the number of classes minus one.

    Returns
    -------
    (np.ndarray, np.ndarray)
        Gradient and Hessian of logistic ordinal negative log likelihood
    """
    grad = gradient_ordinal_logistic_nll(y_true, y_pred, theta)
    hess = hessian_ordinal_logistic_nll(y_true, y_pred, theta)
    return (grad, hess)



class LGBMOrdinal(LGBMRegressor):
    def __init__(
        self,
        #  threshold_interval: float=2,
        boosting_type: str = "gbdt",
        num_leaves: int = 31,
        max_depth: int = -1,
        learning_rate: float = 0.1,
        n_estimators: int = 100,
        subsample_for_bin: int = 200000,
        objective="immediate-thresholds",
        class_weight=None,
        min_split_gain: float = 0.0,
        min_child_weight: float = 1e-3,
        min_child_samples: int = 20,
        subsample: float = 1.0,
        subsample_freq: int = 0,
        colsample_bytree: float = 1.0,
        reg_alpha: float = 0.0,
        reg_lambda: float = 0.0,
        random_state=None,
        n_jobs: int = -1,
        silent="warn",
        importance_type: str = "split",
        **kwargs,
    ):
        super().__init__(
            objective=None,
            boosting_type=boosting_type,
            num_leaves=num_leaves,
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            subsample_for_bin=subsample_for_bin,
            class_weight=class_weight,
            min_split_gain=min_split_gain,
            min_child_weight=min_child_weight,
            min_child_samples=min_child_samples,
            subsample=subsample,
            subsample_freq=subsample_freq,
            colsample_bytree=colsample_bytree,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            random_state=random_state,
            n_jobs=n_jobs,
            silent=silent,
            importance_type=importance_type,
            **kwargs,
        )
        # self.threshold_interval = threshold_interval

    def _initialise_theta(self):
        return np.linspace(0, (self.n_classes - 2) * 1, self.n_classes - 1)

    def _lgb_loss_factory(self):
        self.theta = self._initialise_theta()
        return self.loss  # Return the loss method

    def loss(self, y_test, y_pred):
        # Now `loss` is a class method, not a local function
        return lgb_ordinal_loss(y_test, y_pred, self.theta)

    @staticmethod
    def _alpha_loss_factory(y_true, y_preds):
        """
        Creates loss parametrised by alpha
        """

        def loss(alpha):
            theta = alpha2theta(alpha)
            return ordinal_logistic_nll(y_true=y_true, y_preds=y_preds, theta=theta)

        return loss

    def _optimise_alpha(self, y_true, y_preds):
        """
        Takes loss parametrised by alpha and optimises it.
        Can optionally take in gradient.
        """
        loss = self._alpha_loss_factory(y_true, y_preds)
        alpha = theta2alpha(self.theta)
        bounds = [(None,3.58)]*len(alpha)
        self._alpha_optimisation_report = minimize(loss, alpha, bounds=bounds)
        alpha = self._alpha_optimisation_report.x
        self.theta = alpha2theta(alpha)

    def _initialise_objective(self, y):
        """
        initialises the objective by creating the loss and setting the class
        attributes
        """
        self.n_classes = len(np.unique(y))
        self.objective = self._lgb_loss_factory()
        self._objective = self.objective

    def _output_to_probability(self, output):
        return probas_from_y_pred(output, self.theta)

    def _hot_start(self, X, y, hot_start_iterations=5, **kwargs):
        """
        TODO
        """
        fit_n_estimators = self.n_estimators
        self.n_estimators = hot_start_iterations

        # Fits the model for the default initialisation of alphas
        self._fit(X, y, **kwargs)

        # Updates the alpha to those that minimise the loss
        self._optimise_alpha(y, self.predict_proba(X, raw_score=True))
        self._Booster = None
        self.n_estimators = fit_n_estimators

    def fit(
        self,
        X,
        y,
        hot_start_iterations=5,
        sample_weight=None,
        init_score=None,
        eval_set=None,
        eval_names=None,
        eval_sample_weight=None,
        eval_init_score=None,
        eval_metric=None,
        early_stopping_rounds=None,
        verbose="warn",
        feature_name="auto",
        categorical_feature="auto",
        callbacks=None,
        init_model=None,
    ) -> "LGBMOrdinal":
        """Docstring is inherited from the LGBMModel."""
        self._initialise_objective(y)
        self._hot_start(X, y, hot_start_iterations=hot_start_iterations)
        self._fit(
            X,
            y,
            sample_weight=sample_weight,
            init_score=init_score,
            eval_set=eval_set,
            eval_names=eval_names,
            eval_sample_weight=eval_sample_weight,
            eval_init_score=eval_init_score,
            eval_metric=eval_metric,
            feature_name=feature_name,
            categorical_feature=categorical_feature,
            callbacks=callbacks,
            init_model=init_model,
        )
        return self

    def _fit(
        self,
        X,
        y,
        sample_weight=None,
        init_score=None,
        eval_set=None,
        eval_names=None,
        eval_sample_weight=None,
        eval_init_score=None,
        eval_metric=None,
        early_stopping_rounds=None,
        verbose="warn",
        feature_name="auto",
        categorical_feature="auto",
        callbacks=None,
        init_model=None,
    ) -> "LGBMOrdinal":
        """Docstring is inherited from the LGBMModel."""
        self = super().fit(
            X,
            y,
            sample_weight=sample_weight,
            init_score=init_score,
            eval_set=eval_set,
            eval_names=eval_names,
            eval_sample_weight=eval_sample_weight,
            eval_init_score=eval_init_score,
            eval_metric=eval_metric,
            feature_name=feature_name,
            categorical_feature=categorical_feature,
            callbacks=callbacks,
            init_model=init_model,
        )
        return self

    def predict(
        self,
        X,
        start_iteration=0,
        num_iteration=None,
        pred_leaf=False,
        pred_contrib=False,
        **kwargs,
    ):
        preds = self.predict_proba(
            X,
            raw_score=False,
            start_iteration=start_iteration,
            num_iteration=num_iteration,
            pred_leaf=pred_leaf,
            pred_contrib=pred_contrib,
            **kwargs,
        )
        return np.argmax(preds, axis=1)

    def predict_proba(
        self,
        X,
        raw_score=False,
        start_iteration=0,
        num_iteration=None,
        pred_leaf=False,
        pred_contrib=False,
        **kwargs,
    ):
        preds = super().predict(
            X,
            raw_score=raw_score,
            start_iteration=start_iteration,
            num_iteration=num_iteration,
            pred_leaf=pred_leaf,
            pred_contrib=pred_contrib,
            **kwargs,
        )
        if not raw_score:
            return self._output_to_probability(preds)
        return preds



In [3]:
import polars as pl
import numpy as np
import pandas as pd



In [4]:
def quadratic_weighted_kappa(true: np.ndarray, predicted: np.ndarray) -> np.float64:
    true_binned = true
    predicted_binned = predicted
    
    # Ensure that true_binned and predicted_binned arrays have the same length
    assert len(true_binned) == len(predicted_binned), "The true and predicted arrays must have the same length"
    
    # Define the number of distinct bins
    N = len(np.unique(true_binned))
    
    # Create the histogram matrix O
    O = np.zeros((N, N), dtype=np.float64)
    for t, p in zip(true_binned, predicted_binned):
        O[t, p] += 1
    
    # Create the weight matrix W
    W = np.zeros((N, N), dtype=np.float64)
    for i in range(N):
        for j in range(N):
            W[i, j] = ((i - j) ** 2) / ((N - 1) ** 2)
    
    # Create histograms of the true and predicted values
    true_hist = np.bincount(true_binned, minlength=N)
    pred_hist = np.bincount(predicted_binned, minlength=N)
    
    # Expected matrix E as the outer product of histograms, normalized
    E = np.outer(true_hist, pred_hist) / len(true_binned)
    
    # Normalize O and E to have the same sum
    O_sum = np.sum(O)
    E_sum = np.sum(E)
    
    O_normalized = O / O_sum
    E_normalized = E / E_sum
    
    # Calculate the numerator and denominator of the quadratic weighted kappa
    num = np.sum(W * O_normalized)
    denom = np.sum(W * E_normalized)
    
    # Compute the quadratic weighted kappa
    kappa = 1 - (num / denom)
    
    return kappa

def pre_process_train(main):
    one_hot_columns = ["Basic_Demos-Enroll_Season",
                                    "CGAS-Season",
                                    "Physical-Season",
                                    "Fitness_Endurance-Season",
                                    "FGC-Season",
                                    "BIA-Season",
                                    "PAQ_A-Season",
                                    "PAQ_C-Season",
                                    "PCIAT-Season",
                                    "SDS-Season",
                                    "PreInt_EduHx-Season"]
    one_hot_encoded = pd.get_dummies(main.to_pandas(), columns=one_hot_columns
                                    ,dummy_na=True)
    
    return one_hot_encoded

def pre_process_test(main):
    one_hot_columns = ["Basic_Demos-Enroll_Season",
                                    "CGAS-Season",
                                    "Physical-Season",
                                    "Fitness_Endurance-Season",
                                    "FGC-Season",
                                    "BIA-Season",
                                    "PAQ_A-Season",
                                    "PAQ_C-Season",
                                    "SDS-Season",
                                    "PreInt_EduHx-Season"]
    one_hot_encoded = pd.get_dummies(main.to_pandas(), columns=one_hot_columns
                                    ,dummy_na=True)
    
    return one_hot_encoded

def load_actigraphy_train() -> pl.DataFrame:
    # Load the data
    df = pl.scan_parquet("../input/child-mind-institute-problematic-internet-use/series_train.parquet", hive_partitioning=True)
    non_wear_pct = df.group_by("id").agg(pl.col("non-wear_flag").mean())
    good_data = df.filter(pl.col("non-wear_flag") == 0)
    averages = good_data.group_by("id").agg(
        non_wear_pct=pl.col("non-wear_flag").mean(),
        enmo=pl.col("enmo").mean(),
        peak_enmo=pl.col("enmo").max(),
        anglez=pl.col("anglez").mean(),
        light=pl.col("light").mean(),
        peak_light=pl.col("light").max(),
    )
    averages = averages.join(non_wear_pct, on="id", how="left")

    return averages.collect(streaming=True)

def load_main_train() -> pl.DataFrame:
    # Load the data
    df = pl.read_csv("../input/child-mind-institute-problematic-internet-use/train.csv")
    return df

def load_actigraphy_test() -> pl.DataFrame:
    # Load the data
    df = pl.scan_parquet("../input/child-mind-institute-problematic-internet-use/series_test.parquet", hive_partitioning=True)
    non_wear_pct = df.group_by("id").agg(pl.col("non-wear_flag").mean())
    good_data = df.filter(pl.col("non-wear_flag") == 0)
    averages = good_data.group_by("id").agg(
        non_wear_pct=pl.col("non-wear_flag").mean(),
        enmo=pl.col("enmo").mean(),
        peak_enmo=pl.col("enmo").max(),
        anglez=pl.col("anglez").mean(),
        light=pl.col("light").mean(),
        peak_light=pl.col("light").max(),
    )
    averages = averages.join(non_wear_pct, on="id", how="left")

    return averages.collect(streaming=True)

def load_main_test() -> pl.DataFrame:
    # Load the data
    df = pl.read_csv("../input/child-mind-institute-problematic-internet-use/test.csv")
    return df


def load_train() -> pl.DataFrame:
    avgs = load_actigraphy_train()
    main = pl.DataFrame(pre_process_train(load_main_train()))
    merged = main.join(avgs, on="id", how="left")
    return merged.to_pandas()

def load_test() -> pl.DataFrame:
    avgs = load_actigraphy_test()
    main = pl.DataFrame(pre_process_test(load_main_test()))
    merged = main.join(avgs, on="id", how='left')
    return merged.to_pandas()


In [5]:
data = load_train()
data_test = load_test()


In [6]:

pciat_cols = ['PCIAT-PCIAT_01',
 'PCIAT-PCIAT_02',
 'PCIAT-PCIAT_03',
 'PCIAT-PCIAT_04',
 'PCIAT-PCIAT_05',
 'PCIAT-PCIAT_06',
 'PCIAT-PCIAT_07',
 'PCIAT-PCIAT_08',
 'PCIAT-PCIAT_09',
 'PCIAT-PCIAT_10',
 'PCIAT-PCIAT_11',
 'PCIAT-PCIAT_12',
 'PCIAT-PCIAT_13',
 'PCIAT-PCIAT_14',
 'PCIAT-PCIAT_15',
 'PCIAT-PCIAT_16',
 'PCIAT-PCIAT_17',
 'PCIAT-PCIAT_18',
 'PCIAT-PCIAT_19',
 'PCIAT-PCIAT_20',]
target = data[pciat_cols].dropna()
train = data.dropna(subset=pciat_cols)[data_test.columns]

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import joblib

In [8]:
pciat_out = pl.DataFrame()
pciat_models = {}
for col in pciat_cols:
    y = target[col].values.astype(int)
    X = train.drop('id', axis=1).values
    model = joblib.load(f'../input/pciat-models/{col}.joblib')
    pciat_models[col] = model
    y_hat = model.predict(X)
    print(quadratic_weighted_kappa(y, y_hat))
    pciat_out = pciat_out.with_columns(pl.Series(name=col, values=y_hat))



0.5108713203436928
0.8254459149879478
0.5675071751547364
0.7629324218962888
0.5284183095433783
0.5255786885143026
0.8080040946563061
0.7902768147884178
0.4651564649823169
0.533428880842617
0.5630137726678217
0.8534248241464375
0.7694546816215129
0.7370210067189135
0.7863558228465308
0.7682522028516778
0.5892491986983842
0.452257470298842
0.774021382450578
0.38709344842334603


In [9]:

pciat_out = pciat_out.with_columns(pl.Series(name='PCIAT-TOTAL', values=pciat_out.sum_horizontal().to_numpy()))
pciat_out = pciat_out.with_columns(pl.Series(name='id', values=train['id'].values))

In [10]:
def pciat_to_sii(x):
    if x < 31:
        return 0
    elif x < 49:
        return 1
    elif x < 79:
        return 2
    else:
        return 3

In [11]:
pciat_out

PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-TOTAL,id
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
2,4,4,0,0,0,0,4,0,0,0,0,2,1,2,2,4,2,0,0,27,"""00008ff9"""
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""000fd460"""
2,2,2,0,2,0,0,2,0,0,0,0,0,0,0,0,2,2,0,1,15,"""00105258"""
0,2,2,0,0,0,0,2,0,0,0,0,2,0,0,1,0,0,1,0,10,"""00115b9f"""
2,4,2,0,2,1,0,1,1,0,0,1,2,2,0,0,2,0,1,0,21,"""001f3379"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2,4,2,0,0,0,0,2,0,0,0,0,1,0,0,2,0,0,1,0,14,"""ff6c2bb8"""
0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,3,"""ff759544"""
2,4,4,1,2,1,1,1,1,0,2,0,1,0,2,0,2,2,0,0,26,"""ff8a2de4"""


In [12]:
sii_hat = pciat_out['PCIAT-TOTAL'].map_elements(pciat_to_sii, return_dtype=pl.Int64)

In [13]:
sii_hat.value_counts()

PCIAT-TOTAL,count
i64,u32
2,158
0,2123
3,10
1,380


In [14]:
sii = target[pciat_cols].sum(axis=1).apply(pciat_to_sii)

In [15]:
print(f'Train set QWK: {quadratic_weighted_kappa(sii, sii_hat)}')

Train set QWK: 0.6112416229260772


In [16]:
pciat_test = pl.DataFrame()
for col in pciat_cols:
    X = data_test.drop('id', axis=1).values
    y_hat = pciat_models[col].predict(X)
    pciat_test = pciat_test.with_columns(pl.Series(name=col, values=y_hat))

pciat_test = pciat_test.with_columns(pl.Series(name='PCIAT-TOTAL', values=pciat_test.sum_horizontal().to_numpy()))
pciat_test = pciat_test.with_columns(pl.Series(name='id', values=data_test['id'].values))
pciat_test = pciat_test.with_columns(pl.Series(name='sii', values=pciat_test['PCIAT-TOTAL'].map_elements(pciat_to_sii, return_dtype=pl.Float64)))
output_file = 'submission.csv'
output = pciat_test.select(pl.col('id'), pl.col('sii')).to_pandas()
output['sii'] = output['sii'].astype(int)
output.to_csv(output_file, index=False)